In [1]:
# Import Libraries 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import pickle

# Load Dataset
dataset_path = "../data/IPO_dataset.csv"  # Adjust path as needed
data = pd.read_csv(dataset_path)

# Define Features and Target
features = ['issue_price', 'issue_size', 'hni_subscription', 'nii_subscription',
            'rii_subscription', 'revenue_2', 'revenue_1', 'eps_2', 'eps_1']  # Use only these 9 features
target = 'listing_gain'

# Drop Missing Values
data = data.dropna(subset=features + [target])

# Scale Only Features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data[features])  # Scale features only
y = data[target].values  # Keep target unscaled

# Combine Scaled Features and Target for Sequence Creation
data_scaled = np.hstack((X_scaled, y.reshape(-1, 1)))

# Create Sequences for LSTM
def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps, :-1])  # Features
        y.append(data[i + time_steps, -1])    # Target
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled, time_steps=10)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)  # Predict Listing Gain
])

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train Model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    verbose=1
)

# Save Model and Scaler
model.save("../models/best_lstm_model.h5")
with open("../models/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully!")


Epoch 1/20
3/3 [==============================] - 15s 1s/step - loss: 816.0311 - mae: 21.3022 - val_loss: 1200.8123 - val_mae: 23.0709
Epoch 2/20
3/3 [==============================] - 0s 76ms/step - loss: 809.8721 - mae: 21.2224 - val_loss: 1193.0605 - val_mae: 23.0294
Epoch 3/20
3/3 [==============================] - 0s 70ms/step - loss: 801.8362 - mae: 21.1157 - val_loss: 1183.8148 - val_mae: 22.9988
Epoch 4/20
3/3 [==============================] - 0s 80ms/step - loss: 793.4057 - mae: 21.0060 - val_loss: 1171.7119 - val_mae: 22.9602
Epoch 5/20
3/3 [==============================] - 0s 75ms/step - loss: 780.7628 - mae: 20.8400 - val_loss: 1154.4950 - val_mae: 22.9086
Epoch 6/20
3/3 [==============================] - 0s 73ms/step - loss: 767.8802 - mae: 20.6366 - val_loss: 1131.7623 - val_mae: 22.8507
Epoch 7/20
3/3 [==============================] - 0s 73ms/step - loss: 744.8575 - mae: 20.3596 - val_loss: 1105.5614 - val_mae: 22.8144
Epoch 8/20
3/3 [==============================] -

c:\users\hp\appdata\local\programs\python\python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
